### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn2 import NN2
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.adam import Adam

from mlcvlab.optim import HIDDEN_LAYER_NEURONS

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    y = y.astype(np.int64)
    return x,y

def prepare_data(x, y):
    y = y & 1
    return x, y

def split_train_test(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=25)
    return X_train, X_test, y_train, y_test

def initialize_model(X_train, X_test, y_train, y_test):
    hidden_layer_neurons = HIDDEN_LAYER_NEURONS
    input_layer_neurons = X_train.shape[1]
    layers_dims = [input_layer_neurons + 1, hidden_layer_neurons + 1, 1]
    W = []
    
    # He initialization
    for l in range(1, len(layers_dims)):
        multiplier = np.sqrt(1/(layers_dims[l]*layers_dims[l - 1])) if l == 1 else np.sqrt(2/(layers_dims[l]*layers_dims[l - 1])) 
        W.append(np.transpose(np.random.randn(layers_dims[l], layers_dims[l - 1]) * multiplier))
    
    W0 = W[0]
    W1 = W[1]
    print(f"Size of W0 : {np.shape(W0)}, Size of W1 : {np.shape(W1)}")
    
    two_layer_nn = NN2()
    two_layer_nn.W = [W0, W1]
    two_layer_nn.layers[0].W = W0
    two_layer_nn.layers[1].W = W1
    return two_layer_nn

def train_model(model, X_train, y_train):
    #final_W = SGD(model, X_train, y_train, lr=0.1)
    final_W = Adam(model, X_train, y_train)
    return final_W

def test_model(model, X_test, y_test, final_W):
    model.W = final_W
    model.layers[0].W = final_W[0]
    model.layers[1].W = final_W[1]
    new_test_X = np.concatenate((np.transpose(X_test), -1*np.ones((1, len(X_test)))))
    y_hat = model.nn2(new_test_X)
    y_hat = ((y_hat.T) >= 0.5) * 1
    total = np.sum(y_hat == y_test)
    return total / len(X_test)

### 5. Run the program

In [5]:

#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train = np.array([y_train]).T
y_test = np.array([y_test]).T

#initialize model
model = initialize_model(X_train, X_test, y_train, y_test)

#training model
final_W = train_model(model, X_train, y_train)
print(f"Completed training model - final W : {final_W}")


#testing model
accuracy = test_model(model, X_test, y_test, final_W)
print(f"Completed testing model - Accuracy : {accuracy}")    

Size of W0 : (785, 401), Size of W1 : (401, 1)
122.37098701243914
122.37098701243914
Iteration: 0 completed
118.3358010934252
118.3356811605316
Iteration: 1 completed
113.04567000526079
113.04719633077288
Iteration: 2 completed
104.75110199368193
104.7575827017905
Iteration: 3 completed
97.48159840831065
97.47539237825131
Iteration: 4 completed
88.42025649394442
88.43277398857502
Iteration: 5 completed
93.36504502612911
93.26770776905727
Iteration: 6 completed
86.52900018991818
86.50770303994146
Iteration: 7 completed
86.50016849775213
86.48918267669588
Iteration: 8 completed
84.27895778789366
84.2643098458114
Iteration: 9 completed
85.59776938649532
85.52273468186057
Iteration: 10 completed
82.50248807169963
82.53182041064862
Iteration: 11 completed
82.13363911455737
82.09434929827657
Iteration: 12 completed
81.55526108611986
81.5538641286839
Iteration: 13 completed
80.13209037440856
80.13165877779785
Iteration: 14 completed
80.75888607829611
80.71552685030123
Iteration: 15 completed


49.5402292815879
49.55250393967825
Iteration: 135 completed
49.33468348520694
49.34722556126169
Iteration: 136 completed
49.13044104618306
49.14330441517107
Iteration: 137 completed
48.922390639491994
48.9356569734379
Iteration: 138 completed
48.71271063373367
48.72629431165293
Iteration: 139 completed
48.50609046177026
48.51985210253201
Iteration: 140 completed
48.30132610227311
48.315268194744995
Iteration: 141 completed
48.094503059763795
48.10870933686259
Iteration: 142 completed
47.88390560462877
47.89836699715704
Iteration: 143 completed
47.67491229387765
47.68946895987576
Iteration: 144 completed
47.46906724351891
47.48365522984919
Iteration: 145 completed
47.262847468314774
47.277520911834166
Iteration: 146 completed
47.05589899016699
47.07065437761998
Iteration: 147 completed
46.85111149611196
46.865868191507225
Iteration: 148 completed
46.64803123527052
46.662780644588885
Iteration: 149 completed
46.44635561366224
46.461093533930274
Iteration: 150 completed
46.24309710005372


35.774262695515425
Iteration: 268 completed
35.72849849257238
35.73009838888686
Iteration: 269 completed
35.68100186532422
35.68264448380872
Iteration: 270 completed
35.642673565249126
35.64416853488187
Iteration: 271 completed
35.59473462793469
35.59640564337763
Iteration: 272 completed
35.554120023443986
35.5556767078261
Iteration: 273 completed
35.510375542957796
35.511972787417584
Iteration: 274 completed
35.466071669000605
35.46766266392644
Iteration: 275 completed
35.42494213545232
35.426465809926725
Iteration: 276 completed
35.382011944799835
35.38357104758093
Iteration: 277 completed
35.340106406532676
35.341649411759946
Iteration: 278 completed
35.298921818312735
35.30045933823753
Iteration: 279 completed
35.25689927935837
35.25845578191787
Iteration: 280 completed
35.216199376072055
35.21773502259615
Iteration: 281 completed
35.177699574956264
35.1791993672156
Iteration: 282 completed
35.13423776594255
35.13578243368892
Iteration: 283 completed
35.09646749830289
35.0978717319